In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
import matplotlib.pyplot as plt
import gensim
import collections
import numpy as np
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
from spacy.lang.es import Spanish
from spacy.lang.es.stop_words import STOP_WORDS
import spacy
import pyLDAvis.gensim
import pyLDAvis

In [2]:
df_articles = pd.read_csv('../Data/Data_clean_csv/clean_dataframe.csv')
df_articles.dropna(inplace = True)

In [3]:
stopwords1 = set(stopwords.words('spanish'))
stopwords2 = set(STOP_WORDS)
stopwords = stopwords1.union(stopwords2)
nlp = spacy.load('es_core_news_md')

In [4]:
def cleaner(word):
    word = re.sub(r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*', '', word, flags=re.MULTILINE)
    word = re.sub(r'(?::|;|=)(?:-)?(?:\)|\(|D|P)', "", word)
    word = re.sub(r'ee.uu', 'eeuu', word)
    word = re.sub(r'\#\.', '', word)
    word = re.sub(r'\n', '', word)
    word = re.sub(r',', '', word)
    word = re.sub(r'\-', ' ', word)
    word = re.sub(r'\.{3}', ' ', word)
    word = re.sub(r'a{2,}', 'a', word)
    word = re.sub(r'é{2,}', 'é', word)
    word = re.sub(r'i{2,}', 'i', word)
    word = re.sub(r'ja{2,}', 'ja', word) 
    word = re.sub(r'á', 'a', word)
    word = re.sub(r'é', 'e', word)
    word = re.sub(r'í', 'i', word)
    word = re.sub(r'ó', 'o', word)
    word = re.sub(r'ú', 'u', word)  
    word = re.sub('[^a-zA-Z]', ' ', word)
    list_word_clean = []
    for w1 in word.split(" "):
        if  w1.lower() not in stopwords:
            list_word_clean.append(w1.lower())

    bigram_list = bigram[list_word_clean]
    out_text = lemmatization(" ".join(bigram_list))
    return out_text

In [5]:
def lemmatization(texts, allowed_postags=['NOUN']):
    texts_out = [ token.text for token in nlp(texts) if token.pos_ in 
                 allowed_postags and len(token.text)>2]
    return texts_out

In [6]:
def text_author (df):
    texts = {}
    for author in df['author'].unique():
        texts[author] = (' ').join(list(df[df['author']==author]['body']))
    return texts

In [7]:
def display_topics(model):
    tags = []
    for topic in model.print_topics():
        themes = re.findall( r'\"(.[^"]+).?', topic[1])
        tags.append(themes)
    tags = [tag for tag_list in tags for tag in tag_list]
    repeat = [item for item, count in collections.Counter(tags).items() if count > 3]
    
    for topic_idx, topic in enumerate(model.print_topics()):
        print ("Topic %d:" % (topic_idx+1))
        tags = re.findall( r'\"(.[^"]+).?', topic[1])
        tags = [tag for tag in tags if tag not in repeat]
        print (' '.join(tags), '\n')

In [16]:
def corpus_author (author):
    global bigram
    df_auth = df_articles[df_articles['author']==author]
    bigram = gensim.models.Phrases(df_auth['body'].astype('string').to_list()) 
    df_auth['body'] = df_auth['body'].apply(cleaner)
    dictionary = Dictionary(df_auth['body'].to_list())
    dictionary.compactify()
    dictionary.filter_extremes(no_below=2, no_above=0.97, keep_n=None)
    dictionary.compactify()
    corpus = [dictionary.doc2bow(text) for text in df_auth['body'].to_list()]
    return corpus, dictionary

In [17]:
corpus, dictionary = corpus_author('Valeria Moy')

<ipython-input-16-074e80c8aa92>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_auth['body'] = df_auth['body'].apply(cleaner)


In [18]:
ldamodel = LdaModel(corpus=corpus, num_topics=5, id2word=dictionary)
display_topics(ldamodel)

Topic 1:
pobreza empresas personas mercado forma temas 

Topic 2:
cambio forma mercado nivel cambios 

Topic 3:
gasto pesos presupuesto desarrollo inversion 

Topic 4:
mujeres personas desarrollo productividad forma hombres 

Topic 5:
pobreza ingreso informalidad paises personas ingresos tema 



In [19]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.042033 -0.020247       1        1  36.988533
1     -0.000882  0.001630       2        1  21.430784
4     -0.039562 -0.023605       3        1  15.194060
3     -0.001911  0.042523       4        1  14.054405
0      0.000322 -0.000301       5        1  12.332218, topic_info=            Term        Freq       Total Category  logprob  loglift
495      mujeres  119.000000  119.000000  Default  30.0000  30.0000
44       pobreza  190.000000  190.000000  Default  29.0000  29.0000
570      ingreso  125.000000  125.000000  Default  28.0000  28.0000
156     personas  157.000000  157.000000  Default  27.0000  27.0000
1084     hombres   47.000000   47.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
193        datos   17.147850  206.789466   Topic5  -5.2016  -0.3969
132   desarrollo   15.845272  185.187341   Topic5  -5.2806  -0.3655
317         tema   13.445711  132.716495   Topic5  -5.4448  -0.1966
144    inversion   12.921338  129.417705   Topic5  -5.4846  -0.2112
276       sector   12.715206  145.677240   Topic5  -5.5007  -0.3456

[393 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
175       1  0.332290  acceso
175       2  0.209220  acceso
175       3  0.209220  acceso
175       4  0.147684  acceso
175       5  0.098456  acceso
...     ...       ...     ...
508       1  0.195218   zonas
508       2  0.117131   zonas
508       3  0.390437   zonas
508       4  0.117131   zonas
508       5  0.195218   zonas

[1045 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 5, 4, 1])